In [3]:
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from Classes.Communicator import Communicator

diya_name = "06"
comm = Communicator(diya_name, save_data=True, save_frequency=2, choose_specific_directory=False, verbose=True)

In [4]:
import datetime

x_HP = 0
x_FAN = 0

continue_loop = True
initial_time = datetime.datetime.now()
duration = 11 # seconds

try:
    comm.start()
    comm.send_control_input(x_HP, x_FAN)

    while continue_loop:
        current_time = datetime.datetime.now()
        elapsed_time = current_time - initial_time
        if elapsed_time.total_seconds() > duration:
            continue_loop = False

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    comm.stop()

Communicator startedSubscribed to diya06/rx/

Message sent: x_HP = 0, x_FAN = 0
Received non-standard message: TMP&U	 28.01	 27.92	 30.71	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	State:	1
Received non-standard message: rxBuffer:	HELLO
Received non-standard message: S
Received non-standard message: Command not recognized.
Received non-standard message: State:	-1
Received non-standard message: rxBuffer:	HELLO
Received non-standard message: S
                              T_1    T_2    T_3  x_HP  x_FAN
2024-10-30 13:30:42.510073  28.02  27.94  30.68     0      0
Received non-standard message: TMP&U	 28.02	 27.94	 30.68	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194847	TMP&U	 28.03	 27.95	 30.65	Pel


c:\Users\giaco\Git_Repositories\Semester_Thesis_1\Classes\Communicator.py:89: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  self.measurements = pd.concat([self.measurements, self.last_measurement])
c:\Users\giaco\Git_Repositories\Semester_Thesis_1\Classes\Communicator.py:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.measurements = pd.concat([self.measurements, self.last_measurement])


Data saved to history
Received non-standard message: TMP&U	 28.03	 27.95	 30.65	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194848	TMP&U	 28.03	 27.96	 30.62	Pel
Received non-standard message: TMP&U	 28.03	 27.96	 30.62	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194849	TMP&U	 28.04	 27.97	 30.59	Pel
Data saved to history
Received non-standard message: TMP&U	 28.04	 27.97	 30.59	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194850	TMP&U	 28.05	 27.98	 30.57	Pel
Received non-standard message: TMP&U	 28.05	 27.98	 30.57	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194851	TMP&U	 28.06	 27.99	 30.53	Pel
Data saved to history
Received non-standard message: TMP&U	 28.06	 27.99	 30.53	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194852	TMP&U	 28.07	 28.00	 30.51	Pel
Received non-standard message: TMP&U	 28.07	 28.00	 30.51	INA219	Bus Voltage: 798 --	Shunt Voltage: 1 --	i: 0 --	20000101T194853	TMP&U

In [ ]:
comm.stop()

In [ ]:
import paho.mqtt.client as mqtt

# Define the MQTT broker details
broker = "mqtt.119.ovh"
port = 1883
topic = "diya06"
read_topic = topic + "/rx/"
transmit_topic = topic + "/tx/"
keep_alive = 60

last_message = None

# Callback function when a message is received
def on_message(client, userdata, message):
    last_message = message.payload.decode()
    # print(f"Message received: {last_message} on topic {message.topic}")

    parts = last_message.split()
    if "TMP&U" not in last_message or "Peltier:" not in last_message or "Fan:" not in last_message:
        print("Received incomplete message")
        return
    
    # Parse each component based on its position in the message
    timestamp = parts[0]
    sensor_label = parts[1]
    T_1 = float(parts[2])
    T_2 = float(parts[3])
    T_3 = float(parts[4])

    # Parse the 'Peltier' and 'Fan' values by removing their labels
    peltier_value = int(parts[6])
    fan_value = int(parts[8])

    # Print out parsed values
    print("Timestamp:", timestamp)
    print("Measurements:", T_1, T_2, T_3)
    print("Peltier Value:", peltier_value)
    print("Fan Value:", fan_value)
    print("-------------------------------------------")

def send_control_input(client:mqtt.Client, x_HP:int, x_FAN:int):
    sign_HP = "+" if x_HP >= 0 else "-"

    # Bound control values
    x_HP = max(min(x_HP, 100), -100)
    x_FAN = max(min(x_FAN, 100), 0)

    # Format control values
    x_HP_formatted = f"{sign_HP}{abs(int(x_HP)):03d}"
    x_FAN_formatted = f"{int(x_FAN):03d}"

    message_str = f"S{x_HP_formatted}{x_FAN_formatted}"

    client.publish(transmit_topic, message_str)
    print(f"Message sent: x_HP = {x_HP}, x_FAN = {x_FAN}")


# Create an MQTT client instance
# cself.client = mqtt.Client(api_version=mqtt.CallbackAPIVersion.VERSION2) ##################################3
client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)

# Assign the on_message callback
client.on_message = on_message

# Connect to the broker
client.connect(broker, port, keep_alive)

# Subscribe to the specified topic
client.subscribe(read_topic)

In [ ]:
x_HP = 0
x_FAN = 0

send_control_input(client, x_HP, x_FAN)

client.loop_start()

In [ ]:
client.loop_stop()

In [ ]:
client.disconnect()